# Volumetrics: HCIP calculation

We'll implement the volumetric equation:

$$ V = A \times T \times G \times \phi \times N\!\!:\!\!G \times S_\mathrm{O} \times \frac{1}{B_\mathrm{O}} $$

## Gross rock volume 

$$ \mathrm{GRV} = A \times T $$

Let's make a <bold>function</bold> that computes the GRV.

Now we can just call this function, instead of remembering the equation. (Admittedly, the equation is rather easy to remember in this case!)

It works!

Now we need to compensate for the prospect not being a flat slab of rock &mdash; using the geometric factor. 

We will implement the equations implied by this diagram:

In [ ]:
from IPython.display import Image
Image('http://subsurfwiki.org/images/6/66/Geometric_correction_factor.png', width=600)

We can write a function that implements these relationships:

In [ ]:
def geometric_factor(thick, height, top='slab'):

    ratio = thick / height

    if top == 'round':
        g = -0.6 * ratio + 1
    elif top == 'flat':
        g = -0.3 * ratio + 1
    else:
        g = 1

    return g

In [ ]:
geometric_factor(thick, height=100, top='round')

## Multiple prospects

What if we have multiple prospects? Can we still use our functions?

In [ ]:
import numpy as np

In [ ]:
thicknesses = np.array([10, 25, 15, 5, 100])
heights = np.array([75, 100, 20, 100, 200])

In [ ]:
geometric_factor(thicknesses, heights, top='round')

Awesome, everything just works.

## HC pore volume

We need:

- net:gross &mdash; the ratio of reservoir-quality rock thickness to the total thickness of the interval.
- porosity
- $S_\mathrm{O}$ &mdash; the oil saturation, or proportion of oil to total pore fluid.

In [ ]:
netg = 0.5   # fraction
por = 0.24   # fraction
s_o = 0.8    # fraction

hcpv = netg * por * s_o

### EXERCISE

- Turn this into a function by rearranging the following lines of code:

In [ ]:
"""A function to compute the hydrocarbon pore volume."""
return hcpv
hcpv = netg * por * s_o
def calculate_hcpv(netg, por, s_o):

In [ ]:
# Put your code here:



After rearranging the lines, this should give you `0.096`:

In [ ]:
calculate_hcpv(netg, por, s_o)

## Formation volume factor

Oil shrinks when we produce it, especially if it has high GOR. The FVF, or $B_O$, is the ratio of a reservoir barrel to a stock-tank barrel (25 deg C and 1 atm). Typically the FVF is between 1 (heavy oil) and 1.7 (high GOR).

In [ ]:
fvf = 1.1

## Put it all together

Now we have the components of the volumetric equation:

In [ ]:
hcip = grv * hcpv / fvf
hcip

There are about 6.3 barrels in 1 cubic metre, so let's convert to BOE:

In [ ]:
hcip * 6.2898

In [ ]:
def calculate_hcip(thickness, area, height, top, netg, por, s_o, fvf):
    grv = calculate_grv(thickness, area)
    g = geometric_factor(thickness, height, top)
    grv *= g
    hcpv = calculate_hcpv(netg, por, s_o)
    return grv * hcpv / fvf

In [ ]:
height = 100
top = 'slab'

calculate_hcip(thick, area, height, top, netg, por, s_o, fvf)

## Monte Carlo simulation

We can easily draw randomly from distributions of properties:

In [ ]:
netg = np.random.normal(loc=0.5, scale=0.1, size=100)
por = np.random.normal(loc=0.15, scale=0.025, size=100)

In [ ]:
_ = plt.hist(por)

In [ ]:
import seaborn as sns

sns.distplot(por, rug=True)

In [ ]:
hcpv = calculate_hcpv(netg, por, s_o)
hcpv

In [ ]:
sns.distplot(hcpv)

In [ ]:
hcpv.mean()

The histogram looks a bit ragged, but this is probably because of the relatively small number of samples.

### EXERCISE

1. Compute HCIP with these distributions. Make a histogram of the result in millions of barrels.
1. How does the histogram look if you take 1000 samples instead of 100?
1. Make distributions for some of the other properties, like thickness and FVF.

## Spreadsheet of prospects

In [ ]:
import pandas as pd

uid = "1P2JxXG_jLZ0vx8BlFvm0hD6sBBZH2zU8tk9T-SI27mE"
url = f"https://docs.google.com/spreadsheets/d/{uid}/export?format=csv"

df = pd.read_csv(url)
df.head()

In [ ]:
names = {
    'thickness': 'Thick [m]',
    'area': 'Area [km2]',
    'netg': 'N:G',
    'por': 'phi',
    's_o': 'So',
    'fvf': 'Bo'
}

In [ ]:
def hcip_row(row):
    params = {k: row[v] for k, v in names.items()}
    params['height'] = 1e6
    params['top'] = 'slab'
    hcip = calculate_hcip(**params) * row['GeomFactor']
    return hcip


df['HCIP'] = df.apply(hcip_row, axis=1)

In [ ]:
df.head()

In [ ]:
sns.distplot(df.query('HCIP>5').HCIP)

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2016</p>
</div>